In [1]:
import sys
from pathlib import Path

# Find repo root automatically
p = Path.cwd().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent

ROOT = p
SRC = ROOT / "src"

sys.path.insert(0, str(SRC))

print("Repo root:", ROOT)
print("Added to path:", SRC)


Repo root: /Users/michaelee408/Documents/trap_sim
Added to path: /Users/michaelee408/Documents/trap_sim/src


In [2]:
import laplace
print(laplace.__file__)
print([name for name in dir(laplace) if "laplace" in name.lower() or "solve" in name.lower()])


/Users/michaelee408/Documents/trap_sim/src/laplace.py
['LaplaceResult', 'solve_laplace_tagged']


In [3]:
from laplace import solve_laplace_tagged
from metrics import *
from mesh_io import *


In [4]:
OUT = ROOT / "outputs" / "toy"
OUT.mkdir(parents=True, exist_ok=True)
OUT


PosixPath('/Users/michaelee408/Documents/trap_sim/outputs/toy')

In [5]:
from toy_geometry import create_toy_mesh
from toy_bc import make_toy_facet_tags, toy_boundary_values
from laplace import solve_laplace_tagged

L = 1e-3
domain = create_toy_mesh(L=L, nx=32)

facet_tags = make_toy_facet_tags(domain, L=L, patch_halfwidth=3e-4)
bvals = toy_boundary_values(V_rf=1.0)

phi = solve_laplace_tagged(domain, facet_tags, bvals, degree=1)
phi


ld: warning: duplicate -rpath '/Users/michaelee408/Downloads/ENTER/envs/fenicsx/lib' ignored
ld: warning: duplicate -rpath '/Users/michaelee408/Downloads/ENTER/envs/fenicsx/lib' ignored
ld: warning: duplicate -rpath '/Users/michaelee408/Downloads/ENTER/envs/fenicsx/lib' ignored
ld: warning: duplicate -rpath '/Users/michaelee408/Downloads/ENTER/envs/fenicsx/lib' ignored


LaplaceResult(phi=Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, []), (3,)), 0), Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, [])), 0), V=FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, []), (3,)), 0), Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, [])), bcs_applied={1: 0.0, 2: 1.0, 3: 1.0})

In [9]:
from dolfinx.fem import Function

# Find the first attribute inside `phi` that is a dolfinx Function
u = None
for name in dir(phi):
    if name.startswith("_"):
        continue
    val = getattr(phi, name)
    if isinstance(val, Function):
        u = val
        print("Found solution function as phi.%s" % name)
        break

if u is None:
    raise RuntimeError("Couldn't find a dolfinx.fem.Function inside `phi`. Print dir(phi) or phi.__dict__.")

u


Found solution function as phi.phi


Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, []), (3,)), 0), Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, [])), 0)

In [11]:
from dolfinx.io import XDMFFile
from pathlib import Path

ROOT = Path("/Users/michaelee408/Documents/trap_sim")
OUT = ROOT / "outputs" / "toy"
OUT.mkdir(parents=True, exist_ok=True)

with XDMFFile(domain.comm, str(OUT / "phi.xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(u)

print("Saved to:", OUT / "phi.xdmf")


Saved to: /Users/michaelee408/Documents/trap_sim/outputs/toy/phi.xdmf
